In [1]:
from bs4 import BeautifulSoup as bs
from lxml import html
import requests
import pymongo
from splinter import Browser
import time
import pandas as pd
import numpy as np


## initialize browser and login

In [2]:
# initialize browser
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

### insert profile_id # to search HERE

In [3]:
#define login url (need to add username and password) 
#define site to visit after being logged in
pof_login_url = 'https://login.pof.com'

# update with new profile id here
pof_url = 'https://www.pof.com/viewprofile.aspx?profile_id=76643805'

### input login credentials HERE

In [4]:
browser.visit(pof_login_url) #visit login page

# add credentials
browser.fill("username", "akela_1111") 
browser.fill("password", "plentyoffish")

browser.click_link_by_id("logincontrol_submitbutton") # click submit to enter credentials

In [5]:
browser.visit(pof_url) # browse specified profile

In [6]:
html = browser.html
soup = bs(html, "html.parser")

In [7]:
profile_dict = {}

## username and headline

In [8]:
# use bs and splinter to find username and headline
user = soup.find(id = "username").text
headline = soup.find(id = "headline").text

print(user)
print(headline)

#add to dict
profile_dict["username"] = user
profile_dict["headline"] = headline

dxrdan
Free spirit looking for another


## Info in profile table 1


In [9]:
# use bs and splinter to find first table of profile information
# this is done in parts
# part 1: find header names for first profile table

about_head = soup.find_all("div", class_= "profileheadtitle")
headers = []
for info in about_head:
    headers.append(info.text.replace("\n", ""))
headers

['About',
 'City',
 'Details',
 'Ethnicity',
 'Intent',
 'Education',
 'Personality',
 'Profession']

In [10]:
# part 2: find first half of content (left side of table)
about_content_1 = soup.find_all("div", class_="profileheadcontent")
content = []
for info in about_content_1:
    content.append(info.text.replace("\n\n", "").replace("\n", " "))
len(content)

4

In [11]:
# part 3: find 2nd half of content (right side of table)

about_content_2 = soup.find_all("div", class_="profileheadcontent-narrow")
content_2 = []
for info in about_content_2:
    content_2.append(info.text.replace("\n\n", "").replace("\n", " "))
len(content_2)

4

In [12]:
# part 4: combine both content into one list
all_content = np.array([content, content_2])
all_content_ordered = np.ravel(all_content, order = "F")
all_content_ordered

array(['Smokes Occasionally with Average body type',
       ' San marcos, Texas ',
       '50 year old Male, 6\' 0" (183cm), Catholic', 'Caucasian, Gemini',
       'dxrdan is looking for a relationship.', 'Bachelors Degree',
       'Free Thinker', 'Legal Support'], dtype='<U42')

In [13]:
# define dictionary to hold data in the first profile table. keys = header/title, value = content

profile_info_1 = dict(zip(headers, all_content_ordered))
profile_info_1

{'About': 'Smokes Occasionally with Average body type',
 'City': ' San marcos, Texas ',
 'Details': '50 year old Male, 6\' 0" (183cm), Catholic',
 'Ethnicity': 'Caucasian, Gemini',
 'Intent': 'dxrdan is looking for a relationship.',
 'Education': 'Bachelors Degree',
 'Personality': 'Free Thinker',
 'Profession': 'Legal Support'}

In [14]:
# add to dict
profile_dict["profile_info_1"] = profile_info_1

## Info in profile table 2

In [15]:
about_head_2 = soup.find_all("span", class_="headline")

headers_2 = []
for i in range(9, 25):
    headers_2.append(about_head_2[i].text.replace("\n",""))
    
headers_2

['I am Seeking a',
 'For',
 ' Needs Test',
 'Chemistry',
 'Do you drink?',
 'Do you want children?',
 'Marital Status',
 'Do you do drugs?',
 'Hair Color',
 'Eye Color',
 'Do you have a car?',
 'Do you have children?',
 'Longest Relationship',
 'How ambitious are you?',
 'Pets',
 'Interests']

In [16]:
about_content_3 = soup.find_all("table")[1]("span", class_="txtGrey")
content_3 = []
for info in about_content_3:
    content_3.append(info.text.replace("\n", ""))
content_3

['Woman',
 'Long Term',
 'Not Completed',
 'View his chemistry results',
 'No',
 'Undecided/Open',
 'Single',
 'No',
 'Brown',
 'Blue',
 'Yes',
 'No',
 'Over 10 years',
 'Somewhat Ambitious',
 'No Pets']

In [17]:
profile_info_2 = dict(zip(headers_2, content_3))
profile_info_2

{'I am Seeking a': 'Woman',
 'For': 'Long Term',
 ' Needs Test': 'Not Completed',
 'Chemistry': 'View his chemistry results',
 'Do you drink?': 'No',
 'Do you want children?': 'Undecided/Open',
 'Marital Status': 'Single',
 'Do you do drugs?': 'No',
 'Hair Color': 'Brown',
 'Eye Color': 'Blue',
 'Do you have a car?': 'Yes',
 'Do you have children?': 'No',
 'Longest Relationship': 'Over 10 years',
 'How ambitious are you?': 'Somewhat Ambitious',
 'Pets': 'No Pets'}

In [18]:
# add to dict
profile_dict["profile_info_2"] = profile_info_2

## Interests

In [19]:
# interests
try:
    interests = soup.find(id="interests").text.split("\n")[1:4]
except: 
    interests = "None"
    print("No interests were entered")

interests

['Wildlife', 'Philosophy', 'Firearms']

In [20]:
# add to dict
profile_dict["interests"] = interests

## About Me

In [21]:
about_me = soup.find(id="description").text.replace("\n", "")
about_me_split = about_me.split(" ")
about_me_split


['Live',
 'music,',
 'great',
 'restaurants,',
 'selective',
 'movies,',
 'long',
 'weekend',
 'adventures']

In [22]:
profile_dict["about_me_text"] = about_me
profile_dict["about_me_split"] = about_me_split

In [23]:
#pd.read_html(pof_url) #403 error - Forbidden???

## dictionary with data of profile_1

In [24]:
profile_dict

{'username': 'dxrdan',
 'headline': 'Free spirit looking for another',
 'profile_info_1': {'About': 'Smokes Occasionally with Average body type',
  'City': ' San marcos, Texas ',
  'Details': '50 year old Male, 6\' 0" (183cm), Catholic',
  'Ethnicity': 'Caucasian, Gemini',
  'Intent': 'dxrdan is looking for a relationship.',
  'Education': 'Bachelors Degree',
  'Personality': 'Free Thinker',
  'Profession': 'Legal Support'},
 'profile_info_2': {'I am Seeking a': 'Woman',
  'For': 'Long Term',
  ' Needs Test': 'Not Completed',
  'Chemistry': 'View his chemistry results',
  'Do you drink?': 'No',
  'Do you want children?': 'Undecided/Open',
  'Marital Status': 'Single',
  'Do you do drugs?': 'No',
  'Hair Color': 'Brown',
  'Eye Color': 'Blue',
  'Do you have a car?': 'Yes',
  'Do you have children?': 'No',
  'Longest Relationship': 'Over 10 years',
  'How ambitious are you?': 'Somewhat Ambitious',
  'Pets': 'No Pets'},
 'interests': ['Wildlife', 'Philosophy', 'Firearms'],
 'about_me_tex